In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "xxx"

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install openai==0.28

In [ ]:
!pip install langchain langchain_community tiktoken openai faiss-cpu langchain_openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00


# 1，管道操作符

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

prompt = PromptTemplate.from_template("List 5 {topic}:")
model = OpenAI(model_name='gpt-4o-mini-2024-07-18')  #默认使用的是 gpt-3.5-turbo-instruct
output_parser = CommaSeparatedListOutputParser()

chain = prompt | model #| output_parser

result = chain.invoke({"topic": "fruits"})
print(result)

Sure! Here are five fruits:

1. Apple
2. Banana
3. Orange
4. Strawberry
5. Mango


In [ ]:
model.model_name

'gpt-4o-mini-2024-07-18'

In [ ]:
prompt

PromptTemplate(input_variables=['topic'], template='List 5 {topic}:')

# 2, 字典映射

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# 假设我们已经有了一个向量存储
vectorstore = FAISS.from_texts(["Text about AI", "Text about ML"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template("Context: {context}\nQuestion: {question}")
#model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
)

result = chain.invoke("What is AI?")
print(result)

#AI, or artificial intelligence, refers to the simulation of human intelligence processes by machines, especially computer systems. This can include tasks such as learning, reasoning, problem-solving, perception, and language understanding. AI technologies are designed to mimic human cognitive functions and are used in a wide range of applications, from speech recognition and image processing to autonomous vehicles and medical diagnosis.
# AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are designed to think and act like humans. This can include a variety of capabilities such as learning from experience, understanding natural language, recognizing patterns, and making decisions. AI can be categorized into different types, including narrow AI, which is designed for specific tasks, and general AI, which has the potential to understand and perform any intellectual task that a human can do. Key technologies in AI include machine learning, natural language processing, and robotics.

AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are designed to think and act like humans. AI systems can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, solving problems, and making decisions. These systems are built on algorithms and data, allowing them to learn from experience and improve their performance over time. AI encompasses various subfields, including machine learning, natural language processing, robotics, and computer vision, and is applied in numerous applications such as virtual assistants, autonomous vehicles, and predictive analytics.


In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Context: {context}\nQuestion: {question}'))])

# 3, RunnablePassthrough()

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

prompt = PromptTemplate.from_template("Tell me about {topic}")
model = OpenAI()

chain = RunnablePassthrough() | prompt | model

result = chain.invoke("AI")
print(result)



AI, or artificial intelligence, is the branch of computer science that focuses on creating intelligent machines that can think, learn, and solve problems in a way that mimics human cognition. AI involves the development of algorithms and computer programs that enable machines to process and analyze large amounts of data, recognize patterns, make decisions, and improve their performance over time. Some common applications of AI include speech recognition, image recognition, natural language processing, and robotics. AI has the potential to revolutionize many industries, from healthcare and finance to transportation and manufacturing, by automating tasks and improving efficiency. However, there are also concerns about the ethical implications of creating machines that can think and act like humans, as well as the potential for AI to surpass human intelligence in the future.


In [ ]:
prompt

PromptTemplate(input_variables=['topic'], template='Tell me about {topic}')

# 4, RunnableParallel()

In [ ]:
from langchain.schema.runnable import RunnableParallel
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# 初始化 OpenAI 模型
model = OpenAI()

# 创建提示模板
summarize_prompt = PromptTemplate.from_template("Summarize the following: {text}")
translate_prompt = PromptTemplate.from_template("Translate the following to French: {text}")

# 创建可运行的链
summarize_chain = summarize_prompt | model
translate_chain = translate_prompt | model

# 创建并行运行的链
parallel_chain = RunnableParallel(
    summary=summarize_chain,
    translation=translate_chain
)

# 调用并行链
result = parallel_chain.invoke({"text": "Hello, how are you?"})
print(result)

{'summary': "\n\nThe statement is a greeting and a question about someone's well-being.", 'translation': '\nBonjour, comment ça va ?'}


# 5, RunnableSequence()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = OpenAI(model_name='gpt-4o-mini-2024-07-18')  #默认使用的是 gpt-3.5-turbo-instruct


chain = prompt | model | StrOutputParser()
chain.invoke({"topic": "bears"})


from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()
composed_chain.invoke({"topic": "bears"})

composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)
composed_chain_with_lambda.invoke({"topic": "bears"})

"That's a cute joke! It's lighthearted and unexpected, which can definitely make it funny. Humor often comes from surprise or absurdity, and the image of a bear in a sweater is pretty amusing. Would you like to hear another joke?"

In [ ]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
#sequence = runnable_1 | runnable_2
# Or equivalently:
sequence = RunnableSequence(first=runnable_1, last=runnable_2)
sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

[4, 6, 8]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = OpenAI(model_name='gpt-4o-mini-2024-07-18')  #默认使用的是 gpt-3.5-turbo-instruct


chain = prompt | model | StrOutputParser()
chain.invoke({"topic": "bears"})


from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()
composed_chain.invoke({"topic": "bears"})

composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)
composed_chain_with_lambda.invoke({"topic": "bears"})

"That's a cute and light-hearted joke! It has a playful twist, imagining bears in sweaters and the silliness of it. It's definitely suitable for a good chuckle, especially for kids. Do you have any more jokes like that?"

# 6，RunnableLambda()

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

def uppercase(text):
    return text.upper()

chain = RunnablePassthrough() | RunnableLambda(uppercase)

result = chain.invoke("hello world")
print(result)  # 输出: HELLO WORLD

HELLO WORLD


# 7, RunnableMap()

In [ ]:
from langchain.schema.runnable import RunnableMap

text_processor = RunnableMap({
    "uppercase": lambda x: x.upper(),
    "lowercase": lambda x: x.lower(),
    "length": lambda x: len(x)
})

result = text_processor.invoke("Hello World")
print(result)  # 输出: {'uppercase': 'HELLO WORLD', 'lowercase': 'hello world', 'length': 11}

{'uppercase': 'HELLO WORLD', 'lowercase': 'hello world', 'length': 11}


# 8, RunnableBranch()

In [ ]:
from langchain.schema.runnable import RunnableBranch
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# 初始化 OpenAI 模型
model = OpenAI()

# 创建提示模板
short_prompt = PromptTemplate.from_template("Give a short answer: {input}")
long_prompt = PromptTemplate.from_template("Give a detailed explanation: {input}")

# 创建短答案和长答案链
short_chain = short_prompt | model
long_chain = long_prompt | model

# 创建分支链
chain = RunnableBranch(
    (lambda x: len(x) > 20, long_chain),
    short_chain
)

# 调用链
result_short = chain.invoke("What is AI?")
result_long = chain.invoke("Explain the differences between AI, ML, and deep learning")

print(result_short)
print(result_long)



AI stands for artificial intelligence. It is a branch of computer science that focuses on creating intelligent machines that can think and learn like humans, and perform tasks that typically require human intelligence.


AI (Artificial Intelligence) is a broad field that aims to create intelligent machines that can think and act like humans. It involves developing algorithms and computer programs that can perform tasks that typically require human intelligence, such as problem-solving, decision making, and understanding complex data.

ML (Machine Learning) is a subset of AI that involves training machines to learn from data and improve their performance without being explicitly programmed. It uses statistical techniques to enable machines to identify patterns and make predictions or decisions based on the data they have been given.

Deep Learning is a subfield of ML that uses artificial neural networks to mimic the structure and function of the human brain. It involves feeding large 

# 9, 条件执行

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

model = OpenAI()
formal_prompt = PromptTemplate.from_template("Respond formally: {input}")
casual_prompt = PromptTemplate.from_template("Respond casually: {input}")

def is_formal(text):
    return "sir" in text.lower() or "madam" in text.lower()

def chain(x):
    prompt = formal_prompt if is_formal(x) else casual_prompt
    return model(prompt.format(input=x))

result_formal = chain("Hello sir, how are you?")
result_casual = chain("Hey, what's up?")
print(result_formal)
print(result_casual)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




Hello, I am an AI and do not have the ability to feel emotions. However, thank you for asking. How may I assist you?
 How's your day going?


# 10, 异步支持

In [ ]:
import asyncio
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

model = OpenAI()
prompt = PromptTemplate.from_template("Tell me a fact about {topic}")

chain = prompt | model

async def run_async():
    result = await chain.ainvoke({"topic": "space"})
    print(result)

# Call the async function directly (no need for asyncio.run)
await run_async()



Space is completely silent because sound waves need a medium to travel through, and there is no medium in the vacuum of space. 


# 11，流式处理


In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

model = OpenAI()
prompt = PromptTemplate.from_template("Write a short story about {topic}")

chain = prompt | model

for chunk in chain.stream({"topic": "a brave knight"}):
    print(chunk, end="", flush=True)



Once upon a time in a land far away, there lived a brave knight named Sir William. He was known throughout the kingdom for his courage, loyalty, and strength. Sir William had been a knight since he was a young boy, following in the footsteps of his father who was also a great knight.

One day, the kingdom was suddenly attacked by an evil dragon. The dragon had been terrorizing the kingdom for months, burning villages and causing chaos wherever it went. The king called upon Sir William to defeat the dragon and save the kingdom.

Without hesitation, Sir William mounted his trusty steed and set off to confront the dragon. As he rode through the kingdom, he saw the destruction the dragon had caused. He knew he had to act fast before more innocent lives were lost.

When Sir William finally reached the dragon's lair, he was met with a fierce roar. The dragon was enormous, with scales as black as night and eyes that glowed red with anger. But Sir William did not falter. He drew his sword an

# 12，批处理

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

model = OpenAI()
prompt = PromptTemplate.from_template("Translate '{word}' to French")

chain = prompt | model

words = ["hello", "world", "AI"]
results = chain.batch([{"word": word} for word in words])
for result in results:
    print(result)



bonjour


monde


AI translates to "IA" in French.


# 13，配置和自定义

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

model.invoke("pick a random number")


model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

#AIMessage(content='34')

AttributeError: module 'openai' has no attribute 'OpenAI'

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai ope

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#!pip install --upgrade --quiet  langchain langchain-openai

from langchain_community.chat_models import ChatAnthropic
from langchain_openai import ChatOpenAI

from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")

# Note that we set max_retries = 0 to avoid retrying on RateLimits, etc
openai_llm = ChatOpenAI(max_retries=0)
anthropic_llm = ChatAnthropic()
llm = openai_llm.with_fallbacks([anthropic_llm])

# Let's use just the OpenAI LLm first, to show that we run into an error
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")


#Hit error

In [ ]:
# END